#### With inspiration from 
https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/ 
#### and 
https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

In [30]:
import pandas as pd
import numpy as np

import lightgbm as lgb
from lightgbm import LGBMModel,LGBMClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import metrics
from sklearn.model_selection import cross_val_score

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

from sklearn.model_selection import KFold

In [14]:
# use dataset with encoded categorical cols

train = pd.read_csv("dataset/numerical_train.csv")
train_labels = pd.read_csv("dataset/train_labels.csv")
sub = pd.read_csv("dataset/numerical_sub.csv")

In [15]:
# remove building id

easy_train = train.drop(["building_id"], axis=1)
easy_sub = sub.drop(["building_id"], axis=1)
train_labels = train_labels.drop(["building_id"], axis=1)

In [16]:
# column name formatting 

easy_train.columns = easy_train.columns.str.replace(".", "_")
easy_train = easy_train.drop(['Unnamed: 0'], axis=1)

In [25]:
# combine all training data

train_data = pd.concat([easy_train, train_labels], axis=1)
target = "damage_grade"

In [36]:
# set aside test set

X_train, X_test, y_train, y_test = train_test_split(easy_train, train_labels, test_size=0.15, random_state=42)

In [37]:
# function that helps create lgbm and performs cross-validation

def model_fit(alg, train_set, train_labels, print_feature_importance=True, CV_folds=5):
    
    # set up cross validation data
    kf = KFold(n_splits=CV_folds)
    kf.get_n_splits(train_set)
    
    for train_index, test_index in kf.split(train_set):
        X_train, X_test = train_set[train_index], train_set[test_index]
        y_train, y_test = train_labels[train_index], train_labels[test_index]
        
        print(X_train, X_test)
        print(y_train, y_test)
    
    
#     # fit algorithm on data
#     alg.fit(dtrain[predictors], dtrain[target])
    
#     # predict training set
#     dtrain_predict = alg.predict(dtrain[predictors])
#     dtrain_predprob = alg.predict_proba(dtrain[predictors])[:,1]
    
#     # perform cross-validation
#     if perform_CV:
#         cv_score = cross_val_score(alg, dtrain[predictors], dtrain[target], cv=CV_folds)
        
#     # print model report
#     print("Model Report")
#     print("Accuracy: %.4g" % metrics.accuracy_score(dtrain[target], dtrain_predict))
    
#     if perform_CV:
#         print("CV Score : Mean - %.7g | Std - %.7g | Min - %.7g | Max - %.7g" % (np.mean(cv_score),np.std(cv_score),np.min(cv_score),np.max(cv_score)))
        
#     #Print Feature Importance:
#     if printFeatureImportance:
#         feat_imp = pd.Series(alg.feature_importances_, predictors).sort_values(ascending=False)
#         feat_imp.plot(kind='bar', title='Feature Importances')
#         plt.ylabel('Feature Importance Score')

In [6]:
# split into train test 

X_train, X_test, y_train, y_test = train_test_split(easy_train, train_labels, test_size=0.10, random_state=42)

In [7]:
# creat lgbm classfier

lgb = LGBMClassifier(n_estimators=90, 
                      silent=False, 
                      random_state=43
                     )

In [8]:
# fit with train data 

lgb.fit(X_train, y_train)

/Users/nataliewang/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:235: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/nataliewang/.pyenv/versions/3.7.3/lib/python3.7/site-packages/sklearn/preprocessing/_label.py:268: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=90, n_jobs=-1, num_leaves=31, objective=None,
               random_state=43, reg_alpha=0.0, reg_lambda=0.0, silent=False,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [151]:
# predict 
pred = lgb.predict(X_test)

In [152]:
metrics.f1_score(pred, y_test, average="micro")

0.708031157668547

In [95]:
# predict with lgb

lgb_prediction = lgb.predict(easy_sub)

# Naive Prediction with LGBM

In [93]:
submission = pd.read_csv("dataset/submission_format.csv")

In [94]:
submission.head()

,building_id,damage_grade
0,300051,1
1,99355,1
2,890251,1
3,745817,1
4,421793,1


In [96]:
submission["damage_grade"] = lgb_prediction

In [99]:
submission.to_csv("submissions/naive_lgb_submission.csv", index=False)